In [1]:
from scipy.io import loadmat
import matplotlib.pyplot as plt, numpy as np, numpy.random as npr, csv, matplotlib.pyplot as plt
import torch.nn as nn, timeit, torch, pystan as ps
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import LocallyLinearEmbedding
%matplotlib inline
from time import time
from tqdm import trange
from pylab import plot, show, legend

In [2]:
sm = ps.StanModel(file="/home/postdoc/dsen/Desktop/VAE_density/stan_files/nn_refpoints_constrainedtheta.stan")

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_cf61588642e0539b092dedc55e934141 NOW.


In [3]:
data = loadmat('data.mat')
X = data['data'][:,0:-1].transpose()
Nobs = np.shape(X)[1]

### Local linear embedding for reference points:

In [9]:
n_ref = 500
lat_dim = 1
obs_dim = np.shape(X)[0] 
ref_points = npr.choice(Nobs,n_ref,replace=False)
embedding = LocallyLinearEmbedding(n_components=lat_dim, n_neighbors=10)
X_embedded = embedding.fit_transform(X.transpose()).transpose()

In [10]:
train = np.setdiff1d(np.arange(Nobs), ref_points)
X_ref = X[:,ref_points]
Z_ref = X_embedded[:,ref_points]
X_train = X[:,train]

In [11]:
n_in = np.shape(X_embedded)[0]
n_h = 20
n_out = np.shape(X)[0]

model = nn.Sequential(nn.Linear(n_in, n_h),
                      nn.Tanh(),
                      nn.Linear(n_h, n_out))

In [12]:
def init_normal(m):
    if type(m) == nn.Linear: nn.init.normal_(m.weight)
model.apply(init_normal)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True)

In [13]:
n_iter = 10_000
Lss = np.zeros(n_iter)
start = time()
for epoch in range(n_iter):
    X_pred = model(torch.from_numpy(Z_ref.transpose().astype(np.float32)))
    loss = criterion(X_pred, torch.from_numpy(X_ref.transpose().astype(np.float32)))
    Lss[epoch] = loss.data
    if (epoch+1)%(n_iter/10) == 0 :
        print("Epoch: %d; loss: %.3f; time: %.1f mins" % (epoch+1, loss.item(), (time()-start)/60))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch: 1000; loss: 0.065; time: 0.0 mins
Epoch: 2000; loss: 0.065; time: 0.0 mins
Epoch: 3000; loss: 0.065; time: 0.0 mins
Epoch: 4000; loss: 0.065; time: 0.0 mins
Epoch: 5000; loss: 0.065; time: 0.0 mins
Epoch: 6000; loss: 0.065; time: 0.0 mins
Epoch: 7000; loss: 0.065; time: 0.1 mins
Epoch: 8000; loss: 0.065; time: 0.1 mins
Epoch: 9000; loss: 0.065; time: 0.1 mins
Epoch: 10000; loss: 0.065; time: 0.1 mins


In [14]:
for (i,p) in enumerate(model.parameters()):
    if i == 0 : W_1_optim = p.data;

c = [np.linalg.norm(W_1_optim[i,:]) for i in range(n_in)]
for i in range(n_ref) : Z_ref[:,i] *= c

data = dict(Nobs=Nobs, in_dim=lat_dim, out_dim=obs_dim, hidden_dim=n_h,
            n_ref=np.shape(X_ref)[1], X=X_train, X_ref=X_ref, Z_ref=Z_ref, prior_sigma2=1e-2)

### Run Stan:

In [ ]:
start = time()
fit = sm.sampling(data=data, thin=1, n_jobs=10, chains=10, init="random", iter=1_000)
print(round((time()-start)/60,2), "minutes to run")
print(fit);

In [ ]:
trace = fit.extract()
n_samples, n_latent = np.shape(trace['Z'])[0], np.shape(trace['Z'])[1]

#### Traceplots of latent variables:

In [ ]:
plt.figure(figsize=(20,15))
idx = npr.choice(n_latent,100).reshape(10,10)

for i in range(10):
    for j in range(10):
        k = 10*i+j
        plt.subplot(10,10,k+1)
        plt.plot(trace['Z'][:,idx[i,j],0])
        if i < 9 : plt.xticks(alpha=0)
        if i == 9 : plt.xlabel("MCMC sample", fontsize=12)
        plt.grid(True)
plt.suptitle("Traceplots of some latent variables", fontsize=20, x=0.5,y=0.91)
plt.show()

#### Loss:

In [ ]:
def L2_loss(X, Z, weights_1, bias_1, weights_2, bias_2) :
    X_pred = np.matmul(np.tanh(np.matmul(Z,weights_1)+bias_1), weights_2) + bias_2
    return np.mean(np.sum((X-X_pred)**2,axis=1))

In [ ]:
loss = np.zeros(n_samples)
for i in trange(n_samples) :
    loss[i] = L2_loss(X_train.transpose(), trace['Z'][i], trace['weights_1'][i], 
                      trace['bias_1'][i], trace['weights_2'][i], trace['bias_2'][i])
plt.figure(figsize=(7,4))
plt.plot(loss)
plt.title("Loss", fontsize=15)
plt.xlabel("MCMC sample", fontsize=13)
plt.grid(True)

#### Pairwise distances:

In [ ]:
k_max = 50

pw_dist_matrix = np.zeros((k_max,k_max,n_samples))
for i in trange(k_max) :
    for j in range(k_max) :
        for k in range(n_samples) :
            pw_dist_matrix[i,j,k] = np.linalg.norm(trace['Z'][k,i]-trace['Z'][k,j])

In [ ]:
idx = npr.choice(k_max,10,replace=False)
plt.figure(figsize=(20,15))
for i in range(6):
    for j in range(6):
        if i > j :
            k = 10*i+j+1
            plt.subplot(10,10,k)
            plt.plot(pw_dist_matrix[idx[i],idx[j]])
            if i < 6 : plt.xticks(alpha=0)
            plt.grid(True)
            if i == 5 : plt.xlabel("MCMC sample", fontsize=12)
            if j > 0 : plt.yticks(alpha=0)
plt.suptitle("Traceplots of some pairwise distances", fontsize=15, x=0.31,y=0.83);